In [8]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
import os

In [2]:
batch_size = 32
num_classes = 10
epochs = 100
num_predictions = 20

In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

170500096/170498071 [==============================] - 410s 2us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:
def conv_block(inputs):
    x=Conv2D(32, (3, 3), padding='same')(inputs)
    x=Activation('relu')(x)
    x=Conv2D(32, (3, 3))(x)
    x=Activation('relu')(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Dropout(0.25)(x)

    x=Conv2D(64, (3, 3), padding='same')(x)
    x=Activation('relu')(x)
    x=Conv2D(64, (3, 3))(x)
    x=Activation('relu')(x)
    x=MaxPooling2D(pool_size=(2, 2))(x)
    x=Dropout(0.25)(x)
    conv=Model(inputs, x)
    return conv

In [11]:
def nn_block(inputs):
    x=Flatten()(inputs)
    x=Dense(512)(x)
    x=Activation('relu')(x)
    x=Dropout(0.5)(x)
    x=Dense(num_classes)(x)
    x=Activation('softmax')(x)
    nn=Model(inputs,x)
    return nn

In [36]:
def model_maker():
    a=x_train.shape[1]
    b=x_train.shape[2]
    a=((a-2)//2)-2
    b=((b-2)//2)-2
    nshape=(a//2,b//2,64)
    inputc=Input(shape=x_train.shape[1:])
    inputn=Input(shape=nshape)
    con=conv_block(inputc)
    nn=nn_block(inputn)
    model=Model(inputc,nn(con(inputc)))
    con.trainable=False
    frozen_model=Model(inputc,nn(con(inputc)))
    print(con.summary())
    print(nn.summary())
    print(model.summary())
    print(frozen_model.summary())
    return model, frozen_model

In [37]:
model,f_model=model_maker()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_59 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_60 (Activation)   (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 15, 15, 32)        0  

In [38]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
f_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [39]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [40]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), shuffle=True, callbacks=[tensorboard_callback])
f_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test), shuffle=True, callbacks=[tensorboard_callback])

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 110s 2ms/step - loss: 2.3026 - accuracy: 0.0974 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/100
23232/50000 [============>.................] - ETA: 59s - loss: 2.3026 - accuracy: 0.0979

KeyboardInterrupt: 

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])